In [1]:
# from src.utils import append_gym_checker

# append_gym_checker()

In [1]:
import os
from pathlib import Path
import sys

pwd = Path(os.getcwd())
if (pwd.name == "notebooks"):
    sys.path.append(str(pwd.parent / "gym-checkers-for-thai"))
else:
    sys.path.append(str(pwd / "gym-checkers-for-thai"))

In [2]:
!pwd

/home/boat/pattern/pattern_term_project_2024


In [3]:
from checkers.game import Checkers
from checkers.agents.baselines import play_a_game
from checkers.game import Checkers
from checkers.agents.alpha_beta import MinimaxPlayer, first_order_adv, material_value_adv
from tqdm import tqdm
from functools import partial

In [4]:
from src.prame_q_learn.agent.white_checker_agent import WhiteCheckerAgent
from src.enum import RESULT_TYPE

/home/boat/pattern/pattern_term_project_2024


In [5]:
# import json
# if os.path.exists("src/prame_q_learn/agent/weight/white_weight.json"):
#     os.remove("src/prame_q_learn/agent/weight/white_weight.json")
# open("src/prame_q_learn/agent/weight/white_weight.json","w").write("{}")

In [6]:
# A few matches against a random player
max_game_len = 100
n_matches = 1
n_wins, n_draws, n_losses = 0, 0, 0
is_show_game = True
explore_rate=0.1
for i in tqdm(range(n_matches)):
    if is_show_game:
        print('game', i)
    ch = Checkers()
    black_player = MinimaxPlayer(
        'black',
        # value_func=partial(first_order_adv, 'black', 200, 100, 20, 0),
        value_func=partial(first_order_adv, 'black', 86.0315, 54.568, 87.21072, 25.85066),        
        # The provided legal moves might be ordered differently
        rollout_order_gen=lambda x: sorted(x),
        search_depth=3,
        seed=i)

    # white_player = MinimaxPlayer('white', value_func=partial(material_value_adv, 'white', 2, 1), search_depth=4, seed=i * 2)
    # white_player = RandomPlayer('white', seed=i * 2)
    white_player = WhiteCheckerAgent(weight_path = "src/prame_q_learn/agent/weight/white_weight.json",        
                                    explore_rate=explore_rate,
                                    explore_rate_decay_factor=1.0,
                                    alpha= 0.9,
                                    gamma= 0.5,
                                    win_score= 100,
                                    draw_score= -5,
                                    lose_score= -50,
                                    )

    #modify this function to put our RL model as white
    winner = play_a_game(ch, black_player.next_move, white_player.next_move, max_game_len,is_show_detail = is_show_game)

    # Play with a minimax player
    # play_a_game(ch, keyboard_player_move, white_player.next_move)
    if is_show_game:
        print('black player evaluated %i positions in %.2fs (avg %.2f positions/s) effective branching factor %.2f' % (black_player.n_evaluated_positions, black_player.evaluation_dt, black_player.n_evaluated_positions / black_player.evaluation_dt, (black_player.n_evaluated_positions / black_player.ply) ** (1 / black_player.search_depth)))
        print('black player pruned', black_player.prunes.items())
        print()
    # Keep scores
    # n_wins += 1 if winner == 'black' else 0
    # n_draws += 1 if winner is None else 0
    # n_losses += 1 if winner == 'white' else 0
    result:RESULT_TYPE
    if winner == 'black':
        n_wins += 1
        result = RESULT_TYPE.LOSE
    elif winner is None:
        n_draws += 1
        result = RESULT_TYPE.DRAW
    else:
        n_losses += 1
        result = RESULT_TYPE.WIN
    white_player.get_result(result)
    print(f"round : {i+1}/{n_matches} result {result.value}, explore_rate {explore_rate}")
    
    explore_rate = white_player._explore_rate

print('black win', n_wins, 'draw', n_draws, 'loss', n_losses)

  0%|                                                                                                                                        | 0/1 [00:00<?, ?it/s]

game 0
_b_b_b_b
b_b_b_b_
_._._._.
._._._._
_._._._.
._._._._
_w_w_w_w
w_w_w_w_
0 turn: black last_moved_piece: None
7 legal moves [(4, 8), (5, 8), (5, 9), (6, 9), (6, 10), (7, 10), (7, 11)]
black moved 5, 8
_b_b_b_b
b_._b_b_
_b_._._.
._._._._
_._._._.
._._._._
_w_w_w_w
w_w_w_w_
1 turn: white last_moved_piece: None
7 legal moves [(24, 21), (24, 20), (25, 22), (25, 21), (26, 23), (26, 22), (27, 23)]
white moved 26, 22
_b_b_b_b
b_._b_b_
_b_._._.
._._._._
_._._._.
._._w_._
_w_w_._w
w_w_w_w_
2 turn: black last_moved_piece: None
8 legal moves [(0, 5), (1, 5), (6, 9), (6, 10), (7, 10), (7, 11), (8, 12), (8, 13)]
black moved 7, 11
_b_b_b_b
b_._b_._
_b_._._b
._._._._
_._._._.
._._w_._
_w_w_._w
w_w_w_w_
3 turn: white last_moved_piece: None
8 legal moves [(22, 18), (22, 17), (24, 21), (24, 20), (25, 21), (27, 23), (30, 26), (31, 26)]
white moved 22, 18
_b_b_b_b
b_._b_._
_b_._._b
._._._._
_._._w_.
._._._._
_w_w_._w
w_w_w_w_
4 turn: black last_moved_piece: None
9 legal moves [(0, 5), (1, 5), (2, 7)

100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  1.42it/s]

round : 1/1 result lose, explore_rate 0.1
black win 1 draw 0 loss 0


# Depth N supervised model

## depth 5 vs depth 3 

In [25]:
# A few matches against a random player
max_game_len = 100
n_matches = 1
n_wins, n_draws, n_losses = 0, 0, 0
is_show_game = False
explore_rate=0.1

def rollout_order_gen_random(x):
    random.shuffle(x)
    return x
    
for i in tqdm(range(n_matches)):
    if is_show_game:
        print('game', i)
    ch = Checkers()
    black_player = MinimaxPlayer(
        'black',
        # value_func=partial(first_order_adv, 'black', 200, 100, 20, 0),
        value_func=partial(first_order_adv, 'black', 86.0315, 54.568, 87.21072, 25.85066),        
        # The provided legal moves might be ordered differently
        rollout_order_gen=rollout_order_gen_random,
        search_depth=3,
        seed=i)

    white_player = MinimaxPlayer(
                    'white',
                    # value_func=partial(first_order_adv, 'black', 200, 100, 20, 0),
                    value_func=partial(first_order_adv, 'white', 86.0315, 54.568, 87.21072, 25.85066),        
                    # The provided legal moves might be ordered differently
                    rollout_order_gen=rollout_order_gen_random,
                    search_depth=4,
                    seed=i)

    #modify this function to put our RL model as white
    winner = play_a_game(ch, black_player.next_move, white_player.next_move, max_game_len,is_show_detail = is_show_game)

    # Play with a minimax player
    # play_a_game(ch, keyboard_player_move, white_player.next_move)
    if is_show_game:
        print('black player evaluated %i positions in %.2fs (avg %.2f positions/s) effective branching factor %.2f' % (black_player.n_evaluated_positions, black_player.evaluation_dt, black_player.n_evaluated_positions / black_player.evaluation_dt, (black_player.n_evaluated_positions / black_player.ply) ** (1 / black_player.search_depth)))
        print('black player pruned', black_player.prunes.items())
        print()
    result:RESULT_TYPE
    if winner == 'black':
        n_wins += 1
        result = RESULT_TYPE.LOSE
    elif winner is None:
        n_draws += 1
        result = RESULT_TYPE.DRAW
    else:
        n_losses += 1
        result = RESULT_TYPE.WIN

    print(f"round : {i+1}/{n_matches} result {result.value}, explore_rate {explore_rate}")
    # print(white_player.board_move_dict)

print('black win', n_wins, 'draw', n_draws, 'loss', n_losses)

100%|██████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:05<00:00,  5.08s/it]

round : 1/1 result win, explore_rate 0.1
black win 0 draw 0 loss 1


In [145]:
white_player.board_move_dict[0]

[{'black': {'men': {0, 1, 2, 3, 4, 6, 7, 8}, 'kings': set()},
  'white': {'men': {24, 25, 26, 27, 28, 29, 30, 31}, 'kings': set()}},
 (24, 20)]

In [30]:
import torch
def encode_board_to_feature_map(board):
    output_maps = torch.zeros((4, 8, 4))
    for k,(side, typ) in enumerate([["black","men"], ["black", "kings"], ["white","men"],["white","kings"]]):
        _indices = list(board[side][typ])
        _2d_indices = [[int(i/4), i % 4] for i in _indices]
        _maps = torch.zeros((1,8,4))
        for i,j in _2d_indices:
            _maps[:,i,j] = torch.tensor([1.0])
        output_maps[k, :, :] = _maps
    return output_maps

def encode_action(action):
    output_maps = torch.zeros((2, 8, 4))
    a = [int(action[0] / 4), action[0] % 4]
    b = [int(action[1] / 4), action[1] % 4]
    output_maps[0, a[0],a[1]] = 1.0
    output_maps[1, b[0],b[1]] = 1.0
    return output_maps

In [147]:
encode_board_to_feature_map(white_player.board_move_dict[0][0])

tensor([[[1., 1., 1., 1.],
         [1., 0., 1., 1.],
         [1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [1., 1., 1., 1.],
         [1., 1., 1., 1.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [148]:
encode_action(white_player.board_move_dict[0][1])

tensor([[[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 0.]],

        [[0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.],
         [1., 0., 0., 0.],
         [0., 0., 0., 0.],
         [0., 0., 0., 0.]]])

In [6]:
import torch
import torch.nn as nn
from torch.utils.data import TensorDataset, DataLoader

class MultiLayerDenseModel(nn.Module):
    def __init__(self, input_size, output_size, hidden_sizes):
        super(MultiLayerDenseModel, self).__init__()
        self.input_size = input_size
        self.output_size = output_size
        self.hidden_sizes = hidden_sizes
        
        # Define layers
        layers = []
        in_features = input_size
        for hidden_size in hidden_sizes:
            layers.append(nn.Linear(in_features, hidden_size))
            layers.append(nn.ReLU())  # You can change the activation function if needed
            in_features = hidden_size
        layers.append(nn.Linear(in_features, output_size))
        layers.append(nn.Sigmoid())
        
        # Create the sequential model
        self.model = nn.Sequential(*layers)
        
    def forward(self, x):
        return self.model(x)

# Example usage:
input_size = 128
output_size = 64
hidden_sizes = [256, 512, 256]  # Example hidden layer sizes
model = MultiLayerDenseModel(input_size, output_size, hidden_sizes)
print(model)

MultiLayerDenseModel(
  (model): Sequential(
    (0): Linear(in_features=128, out_features=256, bias=True)
    (1): ReLU()
    (2): Linear(in_features=256, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=256, bias=True)
    (5): ReLU()
    (6): Linear(in_features=256, out_features=64, bias=True)
    (7): Sigmoid()
  )
)


## train loop

In [188]:
# # in_data = encode_board_to_feature_map(white_player.board_move_dict[0][0])
# in_data = in_data[None,:,:,:]
# print(in_data.shape)
# in_data = in_data.flatten(start_dim=1)
# print(in_data.shape)
# model.forward(in_data).shape

In [189]:
loss_fn = torch.nn.MSELoss(reduction="sum")
optimizer = torch.optim.Adam(model.parameters(), lr = 0.0005)

In [ ]:
# A few matches against a random player
max_game_len = 100
n_matches = 1000
n_wins, n_draws, n_losses = 0, 0, 0
is_show_game = False
train_batch_size = 16
device =  "cuda"
offset_seed = 1000
model = model.to(device)
train_seqs = []

for i in tqdm(range(n_matches)):
    if is_show_game:
        print('game', i)
    ch = Checkers()
    black_player = MinimaxPlayer(
        'black',
        # value_func=partial(first_order_adv, 'black', 200, 100, 20, 0),
        value_func=partial(first_order_adv, 'black', 86.0315, 54.568, 87.21072, 25.85066),        
        # The provided legal moves might be ordered differently
        rollout_order_gen=lambda x: sorted(x),
        search_depth=2,
        seed=i+offset_seed)

    white_player = MinimaxPlayer(
                    'white',
                    # value_func=partial(first_order_adv, 'black', 200, 100, 20, 0),
                    value_func=partial(first_order_adv, 'white', 86.0315, 54.568, 87.21072, 25.85066),        
                    # The provided legal moves might be ordered differently
                    rollout_order_gen=lambda x: sorted(x),
                    search_depth=4,
                    seed=i+offset_seed)

    #modify this function to put our RL model as white
    winner = play_a_game(ch, black_player.next_move, white_player.next_move, max_game_len,is_show_detail = is_show_game)

    # Play with a minimax player
    # play_a_game(ch, keyboard_player_move, white_player.next_move)
    if is_show_game:
        print('black player evaluated %i positions in %.2fs (avg %.2f positions/s) effective branching factor %.2f' % (black_player.n_evaluated_positions, black_player.evaluation_dt, black_player.n_evaluated_positions / black_player.evaluation_dt, (black_player.n_evaluated_positions / black_player.ply) ** (1 / black_player.search_depth)))
        print('black player pruned', black_player.prunes.items())
        print()
    result:RESULT_TYPE
    if winner == 'black':
        n_wins += 1
        result = RESULT_TYPE.LOSE
    elif winner is None:
        n_draws += 1
        result = RESULT_TYPE.DRAW
    else:
        n_losses += 1
        result = RESULT_TYPE.WIN

    print(f"round : {i+1}/{n_matches} result {result.value}")

    # training 
    if result.value != "win": continue # dont learn if current episode is not winning

    train_seqs.extend(white_player.board_move_dict)

    if i % 2 != 0: continue
        
    print("start training")
    encoded_boards = []
    encoded_actions = []
    for board, action in train_seqs:
        encoded_boards.append(encode_board_to_feature_map(board))
        encoded_actions.append(encode_action(action))

    encoded_boards = torch.stack(encoded_boards)
    encoded_actions = torch.stack(encoded_actions)

    dataset = TensorDataset(encoded_boards[:encoded_boards.shape[0]//2], encoded_actions[:encoded_boards.shape[0]//2])
    val_dataset = TensorDataset(encoded_boards[encoded_boards.shape[0]//2:], encoded_actions[encoded_boards.shape[0]//2:])

    train_dataloader = DataLoader(dataset, batch_size = train_batch_size, shuffle=True)
    val_dataloader = DataLoader(val_dataset, batch_size = train_batch_size)

    # train loop
    for encoded_board, encoded_action in train_dataloader:
        
        encoded_board = encoded_board.to(device)
        encoded_action = encoded_action.to(device)

        encoded_board = encoded_board.flatten(start_dim = 1)
        encoded_action = encoded_action.flatten(start_dim = 1)
        
        predicted_action = model(encoded_board)

        loss = loss_fn(predicted_action, encoded_action)

        optimizer.zero_grad()
        loss.backward()
        optimizer.step()
        print("train loss:", loss)

    # val loop
    with torch.no_grad():
        val_loss = 0
        count = 0
        for encoded_board, encoded_action in val_dataloader:
                
                encoded_board = encoded_board.to(device)
                encoded_action = encoded_action.to(device)
        
                encoded_board = encoded_board.flatten(start_dim = 1)
                encoded_action = encoded_action.flatten(start_dim = 1)
                
                predicted_action = model(encoded_board)
        
                loss = loss_fn(predicted_action, encoded_action)
            
                val_loss += loss.item()
                count += 1
        print("val loss", val_loss/count)

    train_seqs = []

print('black win', n_wins, 'draw', n_draws, 'loss', n_losses)

  0%|▏                                                                                                                                         | 1/1000 [00:03<1:02:49,  3.77s/it]

round : 1/1000 result win
start training
train loss: tensor(175.6419, device='cuda:0', grad_fn=<MseLossBackward0>)
val loss 174.19525146484375


  0%|▎                                                                                                                                         | 2/1000 [00:07<1:02:59,  3.79s/it]

round : 2/1000 result win


  0%|▍                                                                                                                                         | 3/1000 [00:11<1:03:21,  3.81s/it]

...

round : 107/1000 result win
start training
train loss: tensor(31.9275, device='cuda:0', grad_fn=<MseLossBackward0>)
train loss: tensor(11.9895, device='cuda:0', grad_fn=<MseLossBackward0>)
val loss 21.95674228668213


 11%|██████████████▉                                                                                                                           | 108/1000 [07:01<58:05,  3.91s/it]

round : 108/1000 result win


 11%|███████████████                                                                                                                           | 109/1000 [07:05<58:03,  3.91s/it]

round : 109/1000 result win
start training
train loss: tensor(31.9226, device='cuda:0', grad_fn=<MseLossBackward0>)
train loss: tensor(11.9908, device='cuda:0', grad_fn=<MseLossBackward0>)
val loss 21.954666137695312


In [158]:
# torch.save(model.state_dict(), open("./modelDenseSomthing_latest_2.pt", "wb"))

In [7]:
# dic = torch.load("/home/boat/pattern/pattern_term_project_2024/boat_weight/densenet/modelDenseSomthing_latest_msesum.pt")
dic = torch.load("/home/boat/pattern/pattern_term_project_2024/boat_weight/densenet/modelDenseSomthing_latest_msesum_depth4vs2_randRollout.pt")
model.load_state_dict(dic["model"])
device = "cuda"
model = model.to(device)

In [8]:
dic["iteration"]

9990

In [10]:
from checkers.agents import Player

# A random player
class DenseNetPlayer(Player):
    """A player that makes random legal moves."""
    def __init__(self, color, model, device):
        super().__init__(color=color)
        self.device = device
        self.model = model
        self.model.to(device)
        self.color = color
        
    def next_move(self, board, last_moved_piece):
        state = (board, self.color, last_moved_piece)
        self.simulator.restore_state(state)
        legal_moves = self.simulator.legal_moves()
        # print("legal_moves", legal_moves)

        # inference here
        encoded_board = encode_board_to_feature_map(board)
        encoded_board = encoded_board.flatten(start_dim=0)
        encoded_board = encoded_board[None, :].to(self.device)

        predict = model(encoded_board)[0]

        action_map = predict.reshape((2,8,4)).cpu().detach().numpy()
        
        max_pos_a = np.unravel_index(np.argmax(action_map[0,:,:]), (8,4))
        max_pos_b = np.unravel_index(np.argmax(action_map[1,:,:]), (8,4))

        pred_move = ((max_pos_a[0] * 4) + max_pos_a[1], (max_pos_b[0] * 4) + max_pos_b[1])
        # print(action_map, max_pos_a, max_pos_b)
        # print("pred_move",pred_move)

        # print(legal_moves)

        # if prediction is legal
        p = pred_move
        for legal_move in legal_moves:
            l = legal_move
            if p[0] == l[0] and p[1] == l[1]:
                print("choose predict move", pred_move)
                return pred_move

        # the predict is not legal but predict target position is legal
        for legal_move in legal_moves:
            l = legal_move
            if p[0] == l[0]:
                print("choose legal move", legal_move, "from predict target", pred_move)
                return legal_move

        for legal_move in legal_moves:
            l = legal_move
            if p[1] == l[1]:
                print("choose legal move", legal_move, "from predict target", pred_move)
                return legal_move

        # else: prediction is trash. random
        move = self.random.choice(np.asarray(legal_moves, dtype="int,int"))
        print("random", move)
        
        return move

In [12]:
# A few matches against a random player
import numpy as np
import random

max_game_len = 100
n_matches = 100
n_wins, n_draws, n_losses = 0, 0, 0
is_show_game = False
explore_rate=0.1
rand = 1203

def rollout_order_gen_random(x):
    random.shuffle(x)
    return x
    
for i in tqdm(range(n_matches)):
    if is_show_game:
        print('game', i)
    ch = Checkers()
    black_player = MinimaxPlayer(
        'black',
        # value_func=partial(first_order_adv, 'black', 200, 100, 20, 0),
        value_func=partial(first_order_adv, 'black', 86.0315, 54.568, 87.21072, 25.85066),        
        # The provided legal moves might be ordered differently
        rollout_order_gen=rollout_order_gen_random,
        search_depth=2,
        seed=i+rand)

    white_player = DenseNetPlayer("white",model, device)

    #modify this function to put our RL model as white
    winner = play_a_game(ch, black_player.next_move, white_player.next_move, max_game_len,is_show_detail = is_show_game)

    # Play with a minimax player
    # play_a_game(ch, keyboard_player_move, white_player.next_move)
    if is_show_game:
        print('black player evaluated %i positions in %.2fs (avg %.2f positions/s) effective branching factor %.2f' % (black_player.n_evaluated_positions, black_player.evaluation_dt, black_player.n_evaluated_positions / black_player.evaluation_dt, (black_player.n_evaluated_positions / black_player.ply) ** (1 / black_player.search_depth)))
        print('black player pruned', black_player.prunes.items())
        print()
    result:RESULT_TYPE
    if winner == 'black':
        n_wins += 1
        result = RESULT_TYPE.LOSE
    elif winner is None:
        n_draws += 1
        result = RESULT_TYPE.DRAW
    else:
        n_losses += 1
        result = RESULT_TYPE.WIN

    print(f"round : {i+1}/{n_matches} result {result.value}, explore_rate {explore_rate}")

print('black win', n_wins, 'draw', n_draws, 'loss', n_losses)

  0%|                                                                                                                                         | 0/100 [00:00<?, ?it/s]

choose predict move (26, 22)
choose predict move (25, 21)
choose predict move (24, 20)
choose predict move (21, 16)
choose predict move (22, 18)
choose legal move (18, 15) from predict target (18, 12)
random (31, 26)
choose predict move (26, 23)
choose legal move (23, 19) from predict target (23, 14)
choose predict move (29, 24)
random (30, 26)
random (16, 13)
choose predict move (26, 22)
choose legal move (24, 21) from predict target (30, 21)
choose predict move (21, 17)
choose legal move (17, 14) from predict target (30, 14)
choose predict move (13, 9)
choose legal move (14, 10) from predict target (14, 9)
choose predict move (9, 6)
choose legal move (28, 24) from predict target (28, 15)
choose legal move (24, 21) from predict target (28, 21)
random (19, 15)
random (10, 6)
random (20, 16)


  1%|█▎                                                                                                                               | 1/100 [00:00<00:27,  3.61it/s]

choose legal move (16, 13) from predict target (16, 12)
random (15, 11)
random (6, 2)
random (11, 7)
choose legal move (27, 23) from predict target (20, 23)
choose legal move (2, 6) from predict target (2, 21)
random (6, 15)
round : 1/100 result lose, explore_rate 0.1
choose predict move (26, 22)
choose predict move (25, 21)
choose predict move (21, 17)
choose predict move (22, 18)
choose predict move (24, 20)
choose predict move (17, 14)
choose predict move (14, 9)
choose predict move (18, 14)
choose predict move (29, 24)
choose predict move (30, 21)
choose predict move (21, 16)
choose predict move (24, 17)
choose predict move (17, 8)
choose predict move (27, 23)
choose predict move (23, 19)
choose legal move (31, 27) from predict target (31, 24)
choose legal move (14, 10) from predict target (14, 21)
random (28, 24)
choose predict move (24, 21)
choose predict move (21, 17)
choose predict move (17, 14)
choose predict move (9, 6)
choose predict move (14, 9)
choose legal move (9, 5) fro

  2%|██▌                                                                                                                              | 2/100 [00:00<00:24,  4.06it/s]

...

choose legal move (22, 18) from predict target (22, 21)
choose predict move (25, 21)
choose predict move (21, 16)
choose predict move (24, 17)
choose predict move (17, 14)
choose predict move (14, 9)
choose legal move (18, 14) from predict target (20, 14)
choose predict move (14, 9)
choose predict move (27, 23)
choose predict move (23, 19)
choose predict move (30, 26)
choose predict move (29, 22)
choose legal move (26, 23) from predict target (26, 17)
choose predict move (22, 18)
choose predict move (31, 27)
choose predict move (18, 14)
choose predict move (23, 18)
choose legal move (28, 24) from predict target (28, 15)
choose legal move (18, 15) from predict target (28, 15)
choose legal move (14, 10) from predict target (14, 15)
choose predict move (9, 6)
random (19, 15)
choose legal move (27, 23) from predict target (26, 23)
choose predict move (23, 19)
random (10, 6)
random (19, 15)
random (15, 10)
random (10, 7)
round : 97/100 result lose, explore_rate 0.1
choose predict move (26, 

 98%|█████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████▍  | 98/100 [00:17<00:00,  5.70it/s]

choose legal move (16, 12) from predict target (19, 12)
choose legal move (27, 23) from predict target (27, 18)
choose predict move (30, 23)
choose predict move (23, 19)
choose predict move (19, 10)
choose predict move (29, 22)
choose predict move (22, 17)
choose predict move (17, 13)
choose predict move (20, 13)
choose predict move (13, 9)
choose legal move (9, 6) from predict target (9, 27)
choose legal move (31, 27) from predict target (31, 8)
random (28, 24)
choose predict move (24, 21)
choose predict move (21, 17)
choose legal move (17, 14) from predict target (17, 8)
choose predict move (14, 9)
choose legal move (9, 6) from predict target (9, 23)
choose predict move (27, 23)
choose legal move (23, 18) from predict target (23, 19)
random (6, 2)
choose legal move (2, 6) from predict target (2, 23)
random (6, 9)
choose predict move (9, 6)
choose legal move (6, 1) from predict target (2, 1)
choose predict move (1, 8)
choose legal move (12, 8) from predict target (10, 8)
round : 98/10

100%|███████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 100/100 [00:18<00:00,  5.52it/s]

choose predict move (22, 18)
random (29, 25)
choose predict move (25, 21)
choose legal move (21, 17) from predict target (21, 27)
choose legal move (19, 15) from predict target (19, 26)
choose legal move (20, 16) from predict target (20, 13)
random (12, 8)
random (31, 26)
random (28, 24)
round : 99/100 result lose, explore_rate 0.1
choose predict move (26, 22)
choose legal move (22, 18) from predict target (22, 21)
choose predict move (25, 21)
choose predict move (21, 16)
choose predict move (24, 17)
choose predict move (17, 14)
choose legal move (30, 26) from predict target (30, 23)
choose predict move (29, 22)
choose legal move (22, 18) from predict target (22, 23)
choose legal move (26, 17) from predict target (26, 23)
choose legal move (17, 13) from predict target (17, 23)
choose predict move (13, 9)
choose predict move (14, 9)
choose legal move (27, 23) from predict target (27, 18)
choose predict move (23, 19)
choose predict move (31, 27)
choose legal move (9, 6) from predict targ

# DeepKILLme

In [78]:
# board configuration

num_in_feat = 8*4*4

In [79]:
import torch
import torch.nn as nn

class DeepKILLme(nn.Module):
    def __init__(self, in_feat, out_feat):
        super(DeepKILLme, self).__init__()
        # Define layers
        self.fc1 = nn.Linear(in_feat, 256)  # First fully connected layer
        self.fc2 = nn.Linear(256, 256)          # Second fully connected layer
        self.fc3 = nn.Linear(256, 256)          # Third fully connected layer
        self.fc4 = nn.Linear(256, 512)           # Fourth fully connected layer
        self.fc5 = nn.Linear(512, out_feat)  # Final output layer

        # Define activation function
        self.relu = nn.ReLU()

    def forward(self, x):
        # Forward pass through the network
        x = self.relu(self.fc1(x))
        x = self.relu(self.fc2(x))
        x = self.relu(self.fc3(x))
        x = self.relu(self.fc4(x))
        x = self.fc5(x)
        return x

# Instantiate the deeper model
model = DeepKILLme(num_in_feat, num_out_feat)
print(model)


DeepKILLme(
  (fc1): Linear(in_features=128, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=256, bias=True)
  (fc3): Linear(in_features=256, out_features=256, bias=True)
  (fc4): Linear(in_features=256, out_features=512, bias=True)
  (fc5): Linear(in_features=512, out_features=64, bias=True)
  (relu): ReLU()
)


In [80]:
# white_possible_moves = []
# for i in ["men","king"]:
#     for from_row in range(8):
#         for from_col in range(4):
#             if i == "men":
#                 possible_moves.append([from_row, from_col, from_row - 1, from_col]) # เฉียงซ่าย
#                 possible_moves.append([from_row, from_col, from_row - 1, from_col + 1]) #เฉียงขวา
#             if i == "king":
#                 # upper left
#                 count = 1
#                 while True:
#                     current_row = from_row - count
#                     current_col = from_col - count
                    
#                 diffX = 0
#                 diffY = 0
#                 while from_row + diffX
possible_moves = {}
inv_possible_moves = {}
count = 0
for from_row in range(8):
    for from_col in range(4):
        for to_row in range(8):
            for to_col in range(4):
                if from_row == to_row and from_col == to_col: continue
                possible_moves[f"{from_row},{from_col},{to_row},{to_col}"] = count
                inv_possible_moves[count] = [from_row, from_col, to_row, to_col]
                count+=1

In [81]:
possible_moves

{'0,0,0,1': 0,
 '0,0,0,2': 1,
 '0,0,0,3': 2,
 '0,0,1,0': 3,
 '0,0,1,1': 4,
 '0,0,1,2': 5,
 '0,0,1,3': 6,
 '0,0,2,0': 7,
 '0,0,2,1': 8,
 '0,0,2,2': 9,
 '0,0,2,3': 10,
 '0,0,3,0': 11,
 '0,0,3,1': 12,
 '0,0,3,2': 13,
 '0,0,3,3': 14,
 '0,0,4,0': 15,
 '0,0,4,1': 16,
 '0,0,4,2': 17,
 '0,0,4,3': 18,
 '0,0,5,0': 19,
 '0,0,5,1': 20,
 '0,0,5,2': 21,
 '0,0,5,3': 22,
 '0,0,6,0': 23,
 '0,0,6,1': 24,
 '0,0,6,2': 25,
 '0,0,6,3': 26,
 '0,0,7,0': 27,
 '0,0,7,1': 28,
 '0,0,7,2': 29,
 '0,0,7,3': 30,
 '0,1,0,0': 31,
 '0,1,0,2': 32,
 '0,1,0,3': 33,
 '0,1,1,0': 34,
 '0,1,1,1': 35,
 '0,1,1,2': 36,
 '0,1,1,3': 37,
 '0,1,2,0': 38,
 '0,1,2,1': 39,
 '0,1,2,2': 40,
 '0,1,2,3': 41,
 '0,1,3,0': 42,
 '0,1,3,1': 43,
 '0,1,3,2': 44,
 '0,1,3,3': 45,
 '0,1,4,0': 46,
 '0,1,4,1': 47,
 '0,1,4,2': 48,
 '0,1,4,3': 49,
 '0,1,5,0': 50,
 '0,1,5,1': 51,
 '0,1,5,2': 52,
 '0,1,5,3': 53,
 '0,1,6,0': 54,
 '0,1,6,1': 55,
 '0,1,6,2': 56,
 '0,1,6,3': 57,
 '0,1,7,0': 58,
 '0,1,7,1': 59,
 '0,1,7,2': 60,
 '0,1,7,3': 61,
 '0,2,0,0': 62,
 '

In [82]:
def encode_board(board):
    output_maps = torch.zeros((4, 8, 4))
    for k,(side, typ) in enumerate([["black","men"], ["black", "kings"], ["white","men"],["white","kings"]]):
        _indices = list(board[side][typ])
        _2d_indices = [[int(i/4), i % 4] for i in _indices]
        _maps = torch.zeros((1,8,4))
        for i,j in _2d_indices:
            _maps[:,i,j] = torch.tensor([1.0])
        output_maps[k, :, :] = _maps
    output_maps = output_maps.flatten()
    return output_maps

def encode_action(action):
    onehot = torch.zeros((len(possible_moves)))
    a = [int(action[0] / 4), action[0] % 4]
    b = [int(action[1] / 4), action[1] % 4]
    index = possible_moves[f"{a[0]},{a[1]},{b[0]},{b[1]}"]
    onehot[index] = torch.tensor([1.0])
    return onehot

def decode_action(encoded_action):
    max_id = encoded_action.argmax(dim=0)

    x = inv_possible_moves[max_id.item()]
    
    max_pos_a = (x[0],x[1])
    max_pos_b = (x[2],x[3])

    pred_move = ((max_pos_a[0] * 4) + max_pos_a[1], (max_pos_b[0] * 4) + max_pos_b[1])
    return pred_move

In [101]:
from src.utils import append_gym_checker

append_gym_checker()

import json
import random
from checkers.agents import Player
import copy

class DeepKILLmePlayer(Player):
    def __init__(
        self,
        color,
        memory,
    ) -> None:
        super().__init__(color=color)
        self._color = color

        self.policyModel = DeepKILLme(num_in_feat, len(possible_moves)).to("cuda")
        self.targetModel = DeepKILLme(num_in_feat, len(possible_moves)).to("cuda")
        self.targetModel.load_state_dict(self.policyModel.state_dict())
        # extra
        self.modelName = "DeepKILLme" #use to told the env to push reward and next state for us
        self.memory = memory
        self._epsilon = 0.1
        self._illegal_move_penalty = -120

    def next_move(self, board, last_moved_piece):
        # fix
        global global_epsilon
        self._epsilon = global_epsilon
        state = board, self.color, last_moved_piece
        self.simulator.restore_state(state)
        legal_moves = self.simulator.legal_moves()
        # up to your model
        move = None
        reward = None
        # print(board)
        encoded_board = encode_board(board).to("cuda")
        encoded_pred_action = self.policyModel(encoded_board).to("cuda")
        if random.random() < self._epsilon:
            pred_move = decode_action(encoded_pred_action)

            if any([legal[0]==pred_move[0] and legal[1] == pred_move[1] for legal in legal_moves]):
                move = pred_move
            else:
                move = random.choice(legal_moves)
                reward = self._illegal_move_penalty
        else:
            move = random.choice(legal_moves)

        self.memory.append([copy.deepcopy(board), copy.deepcopy(move), None, reward]) #board, action, next board, reward (next board will be fill by env later) (reward will fill by model if illegal move, else fill by model)
        
        return move

/home/boat/pattern/pattern_term_project_2024


## Playzone

In [110]:
from torch import optim

BATCH_SIZE = 16
GAMMA = 0.99
EPS_START = 0.9
EPS_END = 0.05
EPS_DECAY = 1000
TAU = 0.005
LR = 1e-4

global_epsilon = 0.9

white_player = DeepKILLmePlayer("white", memory)

memory = []
MAX_MEM_CAPACITY = 10000
optimizer = optim.AdamW(white_player.policyModel.parameters(), lr=LR, amsgrad=True)

In [ ]:
# A few matches against a random player
import numpy as np
import random
import math

max_game_len = 100
n_matches = 10000
n_wins, n_draws, n_losses = 0, 0, 0
is_show_game = False
explore_rate=0.1
rand = 1203

def rollout_order_gen_random(x):
    random.shuffle(x)
    return x
    
for i in tqdm(range(n_matches)):
    if is_show_game:
        print('game', i)
    ch = Checkers()
    black_player = MinimaxPlayer(
        'black',
        # value_func=partial(first_order_adv, 'black', 200, 100, 20, 0),
        value_func=partial(first_order_adv, 'black', 86.0315, 54.568, 87.21072, 25.85066),        
        # The provided legal moves might be ordered differently
        rollout_order_gen=rollout_order_gen_random,
        search_depth=2,
        seed=i+rand)

    #modify this function to put our RL model as white
    winner = play_a_game(ch, black_player.next_move, white_player.next_move, max_game_len,is_show_detail = is_show_game, white_player=white_player)

    # impl train here
    if  white_player.memory.__len__() < BATCH_SIZE: continue
    # clip memory
    if white_player.memory.__len__() > MAX_MEM_CAPACITY:
        white_player.memory = white_player.memory[white_player.memory.__len__() - MAX_MEM_CAPACITY:]
    print("white_player.memory.__len__()",white_player.memory.__len__())

    # filter none (maybe in case of multiple capture, the condition in baseline.py is not cover this case) TODO: impl multiple capture support
    white_player.memory = list(filter(lambda x: x[2] is not None, white_player.memory, ))

    transitions = random.sample(white_player.memory, BATCH_SIZE)

    state_batch = []
    next_state_batch = []
    action_batch = []
    reward_batch = []
    for board, action, next_board, reward in transitions:
        encoded_board = encode_board(board)
        encoded_next_board = encode_board(next_board)
        encoded_action = encode_action(action)
        reward = torch.tensor(reward)

        state_batch.append(encoded_board)
        next_state_batch.append(encoded_next_board)
        action_batch.append(encoded_action)
        reward_batch.append(reward)

    state_batch = torch.stack(state_batch).to("cuda")
    next_state_batch = torch.stack(next_state_batch).to("cuda")
    action_batch = torch.stack(action_batch).to("cuda")
    reward_batch = torch.stack(reward_batch).to("cuda")
    
    print("len batch", len(state_batch), len(next_state_batch), len(action_batch), len(reward_batch))

    # forward 
    state_action_values = white_player.policyModel(state_batch)
    next_state_values = white_player.targetModel(next_state_batch).max(1).values

    # compute expected Q
    expected_state_action_values = (next_state_values * GAMMA) + reward_batch

    criterion = nn.SmoothL1Loss()
    loss = criterion(state_action_values, expected_state_action_values.unsqueeze(1))

    optimizer.zero_grad()
    loss.backward()
    
    # In-place gradient clipping
    torch.nn.utils.clip_grad_value_(white_player.policyModel.parameters(), 100)
    optimizer.step()

    # update target model using policy weight
    target_net_state_dict = white_player.targetModel.state_dict()
    policy_net_state_dict = white_player.policyModel.state_dict()
    for key in policy_net_state_dict:
        target_net_state_dict[key] = policy_net_state_dict[key]*TAU + target_net_state_dict[key]*(1-TAU)
    white_player.targetModel.load_state_dict(target_net_state_dict)

    # decay eps
    global_epsilon = EPS_END + (EPS_START - EPS_END) * \
        math.exp(-1. * i / EPS_DECAY)

    # Play with a minimax player
    # play_a_game(ch, keyboard_player_move, white_player.next_move)
    if is_show_game:
        print('black player evaluated %i positions in %.2fs (avg %.2f positions/s) effective branching factor %.2f' % (black_player.n_evaluated_positions, black_player.evaluation_dt, black_player.n_evaluated_positions / black_player.evaluation_dt, (black_player.n_evaluated_positions / black_player.ply) ** (1 / black_player.search_depth)))
        print('black player pruned', black_player.prunes.items())
        print()
        
    result:RESULT_TYPE
    if winner == 'black':
        n_wins += 1
        result = RESULT_TYPE.LOSE
    elif winner is None:
        n_draws += 1
        result = RESULT_TYPE.DRAW
    else:
        n_losses += 1
        result = RESULT_TYPE.WIN

    print(f"round : {i+1}/{n_matches} result {result.value}, explore_rate {explore_rate}")
    print("loss", loss)
    print("memory reward sum", sum([i[3] for i in white_player.memory]))

print('black win', n_wins, 'draw', n_draws, 'loss', n_losses)

  0%|                                                                                                                            | 2/10000 [00:00<24:34,  6.78it/s]

white_player.memory.__len__() 21
len batch 16 16 16 16
round : 1/10000 result lose, explore_rate 0.1
loss tensor(115.7459, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)
memory reward sum -2490
white_player.memory.__len__() 40
len batch 16 16 16 16
round : 2/10000 result lose, explore_rate 0.1
loss tensor(113.8697, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)
memory reward sum -4710


  0%|                                                                                                                            | 4/10000 [00:00<26:52,  6.20it/s]

white_player.memory.__len__() 69
len batch 16 16 16 16
round : 3/10000 result lose, explore_rate 0.1
loss tensor(117.6195, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)
memory reward sum -8190
white_player.memory.__len__() 98
len batch 16 16 16 16
round : 4/10000 result lose, explore_rate 0.1
loss tensor(121.3323, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)
memory reward sum -11640


  0%|                                                                                                                            | 6/10000 [00:00<22:20,  7.45it/s]

...

 26%|███████████████████████████████▏                                                                                         | 2575/10000 [05:52<16:17,  7.60it/s]

white_player.memory.__len__() 10000
len batch 16 16 16 16
round : 2574/10000 result lose, explore_rate 0.1
loss tensor(45.8728, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)
memory reward sum -555830
white_player.memory.__len__() 10000
len batch 16 16 16 16
round : 2575/10000 result lose, explore_rate 0.1
loss tensor(60.8994, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)
memory reward sum -555980


 26%|███████████████████████████████▏                                                                                         | 2577/10000 [05:52<16:30,  7.50it/s]

white_player.memory.__len__() 10000
len batch 16 16 16 16
round : 2576/10000 result lose, explore_rate 0.1
loss tensor(47.3612, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)
memory reward sum -556050
white_player.memory.__len__() 10000
len batch 16 16 16 16
round : 2577/10000 result lose, explore_rate 0.1
loss tensor(62.4181, device='cuda:0', grad_fn=<SmoothL1LossBackward0>)
memory reward sum -556090


 26%|███████████████████████████████▏                                                                                         | 2578/10000 [05:52<16:15,  7.60it/s]

In [76]:
# encode_board_to_feature_map()

In [94]:
white_player.memory

[[{'black': {'men': {0, 1, 2, 3, 4, 5, 6, 11}, 'kings': set()},
   'white': {'men': {24, 25, 26, 27, 28, 29, 30, 31}, 'kings': set()}},
  (26, 23),
  {'black': {'men': {0, 1, 2, 3, 4, 5, 6, 11}, 'kings': set()},
   'white': {'men': {23, 24, 25, 27, 28, 29, 30, 31}, 'kings': set()}},
  -120],
 [{'black': {'men': {0, 1, 2, 3, 4, 6, 8, 11}, 'kings': set()},
   'white': {'men': {23, 24, 25, 27, 28, 29, 30, 31}, 'kings': set()}},
  (23, 18),
  {'black': {'men': {0, 1, 2, 3, 4, 6, 8, 11}, 'kings': set()},
   'white': {'men': {18, 24, 25, 27, 28, 29, 30, 31}, 'kings': set()}},
  -120],
 [{'black': {'men': {0, 1, 2, 3, 4, 6, 11, 12}, 'kings': set()},
   'white': {'men': {18, 24, 25, 27, 28, 29, 30, 31}, 'kings': set()}},
  (18, 15),
  {'black': {'men': {0, 1, 2, 3, 4, 6, 11, 12}, 'kings': set()},
   'white': {'men': {15, 24, 25, 27, 28, 29, 30, 31}, 'kings': set()}},
  -150],
 [{'black': {'men': {0, 1, 2, 3, 4, 6, 12, 18}, 'kings': set()},
   'white': {'men': {24, 25, 27, 28, 29, 30, 31}, 'kin

In [98]:
[i for i in white_player.memory if not i[2]]

[[{'black': {'men': {0, 1, 2, 3, 4, 11, 14, 21}, 'kings': set()},
   'white': {'men': {22, 23, 24, 26, 28, 29, 31}, 'kings': set()}},
  (24, 17),
  None,
  -150]]